In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree, model_selection
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

import re

import numpy as np
import json
import io
import datetime as dt
import string
import unicodedata

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

from gensim import corpora

%pylab --no-import-all inline

C:\Users\gemin\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array
C:\Users\gemin\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('Tweets.csv')

In [3]:
print(len(df))
df.head()

14640


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
# Find NaN values in each column
df.isna().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [5]:
# Are there tweets with negative label but no negative reason?
# This would help us in finding if negative label is a unique identifier for negative tweets or not.
neg_df = df.loc[df['airline_sentiment']=='negative']
neg_df.isna().sum()

tweet_id                           0
airline_sentiment                  0
airline_sentiment_confidence       0
negativereason                     0
negativereason_confidence          0
airline                            0
airline_sentiment_gold          9146
name                               0
negativereason_gold             9146
retweet_count                      0
text                               0
tweet_coord                     8515
tweet_created                      0
tweet_location                  3142
user_timezone                   3170
dtype: int64

In [6]:
# Are there tweets with non negative label but negative reason
pos_df = df.loc[df['airline_sentiment']!='negative']
print(len(pos_df))
pos_df.isna().sum()

5462


tweet_id                           0
airline_sentiment                  0
airline_sentiment_confidence       0
negativereason                  5462
negativereason_confidence       4118
airline                            0
airline_sentiment_gold          5454
name                               0
negativereason_gold             5462
retweet_count                      0
text                               0
tweet_coord                     5106
tweet_created                      0
tweet_location                  1591
user_timezone                   1650
dtype: int64

In [7]:
# Class Distribution or prediction variable
df['airline_sentiment'].value_counts()/len(df)

negative    0.626913
neutral     0.211680
positive    0.161407
Name: airline_sentiment, dtype: float64

In [8]:
#remove any rows that has no tweet text
dataset=df.text.dropna()
print('Number of observations are: '+str(len(dataset)))

Number of observations are: 14640


In [9]:
df['tweet_length'] = df['text'].apply(lambda x : len(x))

In [10]:
df['num_hashtags'] = df['text'].apply(lambda x : len(re.compile(r"#(\w+)").findall(x)))

In [11]:
df['num_mentions'] = df['text'].apply(lambda x : len(re.compile(r"@(\w+)").findall(x)))

In [12]:
# Check if there is significant difference in the length of tweet across different sentiments
# Check if there is significant difference in the number of hashtags across different sentiments
# Check if there is significant difference in the number of mentions across different sentiments

mean_df = df.groupby('airline_sentiment').mean()
mean_df[['tweet_length','num_hashtags', 'num_mentions']]

# Result - not siginificant

,tweet_length,num_hashtags,num_mentions
airline_sentiment,,,
negative,113.947919,0.233384,1.111244
neutral,87.359471,0.212004,1.167473
positive,86.082945,0.297503,1.138383


In [13]:
#Create dictionary of all tweets
tweet_dictionary = {}
i = 0
for line in dataset:
        tweet_dictionary[i] = line.lower()
        i += 1

In [14]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

In [15]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_links(tweet_dictionary[i])

In [16]:
def strip_mentions_hashtags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [17]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_mentions_hashtags(tweet_dictionary[i])

In [18]:
for i in range(0,len(df)):
    tweet_dictionary[i] = tweet_dictionary[i].replace('RT', '')

In [19]:
#code source is http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "youyou will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text.lower())
    return text

In [20]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=expandContractions(tweet_dictionary[i])


In [21]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [22]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=remove_special_characters(tweet_dictionary[i], 
                          remove_digits=True)

In [23]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [24]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=remove_stopwords(tweet_dictionary[i])

In [25]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [26]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [27]:
for i in range(0,len(tweet_dictionary)):
#     tweet_dictionary[i]=simple_stemmer(tweet_dictionary[i])
    tweet_dictionary[i]=lemmatize_text(tweet_dictionary[i])

In [28]:
df['text'] = tweet_dictionary.values()

In [29]:
# Remove words which are shorter than 3 characters. 
# This is for removing words with less meaning
df['text'] = df['text'].apply(lambda x : re.sub(r'\b\w{1,3}\b', '', x))

In [30]:
# TfIdfVectorizer:
# Remove words existing in less than min_df documents and remove words existing in more than max_df percentage of documents
vectorizer = TfidfVectorizer(min_df=15, max_df=0.6, ngram_range=(1,3))
temp_df = vectorizer.fit_transform(df['text'])

In [31]:
text_df = pd.DataFrame(temp_df.toarray(), columns=vectorizer.get_feature_names())

In [33]:
df.groupby('airline_sentiment').mean()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count,tweet_length,num_hashtags,num_mentions
airline_sentiment,,,,,,,
negative,5.692602e+17,0.933365,0.731769,0.093375,113.947919,0.233384,1.111244
neutral,5.691841e+17,0.823303,0.000000,0.060987,87.359471,0.212004,1.167473
positive,5.691006e+17,0.872039,0.000000,0.069403,86.082945,0.297503,1.138383


In [34]:
df = df[['airline_sentiment','retweet_count', 'airline']]

In [35]:
airline_one_hot = pd.get_dummies(df['airline'])
df = df.drop('airline',axis=1)
df = df.join(airline_one_hot)

In [36]:
final_df = pd.concat([df, text_df], axis=1)

In [37]:
final_df.head()

,airline_sentiment,retweet_count,American,Delta,Southwest,US Airways,United,Virgin America,able,absolute,...,would like,would love,would nice,write,wrong,yeah,year,yell,yesterday,zero
0,neutral,0,0,0,0,0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,positive,0,0,0,0,0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,neutral,0,0,0,0,0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,negative,0,0,0,0,0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,negative,0,0,0,0,0,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# Separate features and target
df_features = final_df.iloc[:, 1:].values
df_target = final_df.iloc[:,0].values

In [39]:
clf = tree.DecisionTreeClassifier()
# K-fold construction
folds = 10
kf = model_selection.KFold(n_splits=folds, shuffle=True) 

In [40]:
# K-fold cross validation and performance evaluation
foldid = 0
totacc = 0.
ytlog = []
yplog = []
for train_index, test_index in kf.split(df_features):
    foldid += 1
    print("Starting Fold %d" % foldid)
    print("\tTRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = df_features[train_index], df_features[test_index]
    y_train, y_test = df_target[train_index], df_target[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_pred, y_test)
    totacc += acc
    ytlog += list(y_test)
    yplog += list(y_pred)
    
    print('\tPrediction: ', y_pred)
    print('\tCorrect:    ', y_test)
    print('\tAccuracy:', acc)
    print(classification_report(y_test, y_pred))

print("Average Accuracy: %0.3f" % (totacc / folds,))
print(classification_report(ytlog, yplog))


Starting Fold 1
	TRAIN: 13176 TEST: 1464
	Prediction:  ['positive' 'neutral' 'negative' ..., 'positive' 'negative' 'negative']
	Correct:     ['positive' 'negative' 'negative' ..., 'neutral' 'negative' 'neutral']
	Accuracy: 0.707650273224
              precision    recall  f1-score   support

    negative       0.82      0.80      0.81       933
     neutral       0.47      0.54      0.50       321
    positive       0.61      0.58      0.59       210

   micro avg       0.71      0.71      0.71      1464
   macro avg       0.64      0.64      0.64      1464
weighted avg       0.72      0.71      0.71      1464

Starting Fold 2
	TRAIN: 13176 TEST: 1464
	Prediction:  ['positive' 'neutral' 'neutral' ..., 'negative' 'negative' 'negative']
	Correct:     ['negative' 'positive' 'positive' ..., 'positive' 'negative' 'negative']
	Accuracy: 0.677595628415
              precision    recall  f1-score   support

    negative       0.81      0.77      0.79       933
     neutral       0.44      0.51

In [41]:
# Using Random Forest Classifier
clf = RandomForestClassifier()
# K-fold construction
folds = 5
kf = model_selection.KFold(n_splits=folds, shuffle=True) 

In [42]:
# K-fold cross validation and performance evaluation
foldid = 0
totacc = 0.
ytlog = []
yplog = []
for train_index, test_index in kf.split(df_features):
#     print(train_index)
    foldid += 1
    print("Starting Fold %d" % foldid)
    print("\tTRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = df_features[train_index], df_features[test_index]
    y_train, y_test = df_target[train_index], df_target[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_pred, y_test)
    totacc += acc
    ytlog += list(y_test)
    yplog += list(y_pred)
    
    print('\tPrediction: ', y_pred)
    print('\tCorrect:    ', y_test)
    print('\tAccuracy:', acc)
    print(classification_report(y_test, y_pred))

print("Average Accuracy: %0.3f" % (totacc / folds,))
print(classification_report(ytlog, yplog))


Starting Fold 1
	TRAIN: 11712 TEST: 2928


C:\Users\gemin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


	Prediction:  ['positive' 'neutral' 'neutral' ..., 'negative' 'negative' 'negative']
	Correct:     ['positive' 'negative' 'positive' ..., 'negative' 'neutral' 'neutral']
	Accuracy: 0.731215846995
              precision    recall  f1-score   support

    negative       0.78      0.89      0.83      1825
     neutral       0.54      0.42      0.47       618
    positive       0.72      0.54      0.62       485

   micro avg       0.73      0.73      0.73      2928
   macro avg       0.68      0.62      0.64      2928
weighted avg       0.72      0.73      0.72      2928

Starting Fold 2
	TRAIN: 11712 TEST: 2928
	Prediction:  ['neutral' 'negative' 'positive' ..., 'negative' 'positive' 'negative']
	Correct:     ['positive' 'neutral' 'positive' ..., 'positive' 'positive' 'neutral']
	Accuracy: 0.727459016393
              precision    recall  f1-score   support

    negative       0.78      0.88      0.83      1806
     neutral       0.54      0.45      0.49       640
    positive       0.6